In [0]:
# MAGIC %load_ext autoreload
# MAGIC %autoreload 2

In [0]:
import sys 
import os 
import pyspark.sql.functions as f 
sys.path.append(os.path.abspath('../..'))

In [0]:
from data_loaders.data_load_excel import load_raw_data

In [0]:
data_path = f"/Volumes/pei/default/landing_volume/customers/"
processed_path = f"/Volumes/pei/default/landing_volume/customers/processed/"
file_name = "Customer.xlsx"
full_path = os.path.join(data_path, file_name)

if os.path.exists(full_path):
    try: 
        customer_df = load_raw_data(spark_session=spark, file_path=full_path)

        (
            customer_df
            .withColumn("file_path", f.lit(full_path)) 
            .withColumn("ingestion_timestamp", f.current_timestamp() )
            .write
            .format("delta")
            .option("mergeSchema", "true")
            .mode("append")
            .saveAsTable("pei.bronze.raw_customers")
        )

        dbutils.fs.mv(full_path, os.path.join(processed_path, file_name))
    except Exception as e: 
        print(f"FAILED: Customers Ingestion (Excel). Error: {str(e)}")
else:
    print(f"SKIPPED: {file_name} not found in {data_path}")